In [ ]:
from fitparse import FitFile

In [ ]:
fitfile = FitFile('./2017-11-03-15-58-49.fit')

In [ ]:
fitfile.parse()

In [ ]:
messages = fitfile.messages
print '{} messages'.format(len(messages))

In [ ]:
for i in range(0,10):
    msg = messages[i]
    print 'message no. {}'.format(i)
    print msg.name
    print msg.get_values()

In [ ]:
field_samples = {}
for msg in messages:    
    field_samples[msg.name] = msg
    
print field_samples 

In [ ]:
for k in field_samples:
    print 'Message type: {}'.format(k)
    print field_samples[k].get_values()
    print ''

In [ ]:
field_counts = {}
for msg in messages:
    if msg.name in field_counts:
        count = field_counts[msg.name]
        field_counts[msg.name] = count + 1
    else:
        field_counts[msg.name] = 1


In [ ]:
for k in field_counts:
    print 'record type: {}, count: {}'.format(k, field_counts[k])

In [ ]:
# Show the file type is activity - see section 9 in the FIT file doc for details
fid = field_samples['file_id']
print fid.get_value('type')

In [ ]:
# Show the activity
activity = field_samples['activity']
print activity.get_values()

In [ ]:
# Dump all 'non-record' records
for msg in messages:
    if msg.name != 'record' and msg.name != 'unknown_22':
        print 'Message type: {}'.format(msg.name)
        print msg.get_values()
        print ''

In [ ]:
# Next - extract session start, session end, laps
for msg in messages:
    if msg.name == 'session' or msg.name == 'lap' or msg.name == 'event':
        print 'Message type: {}'.format(msg.name)
        print msg.get_values()
        print ''

In [ ]:
# Get laps
laps = []
for msg in messages:
    if msg.name == 'lap':
        laps.append(msg)
        
start_times = map(lambda x:x.get_value('start_time'), laps)
print start_times

In [ ]:
print laps[0].get_values()

lap_time_sum = 0
for lap in laps:
    lap_time_sum += lap.get_value('total_timer_time')
    
print lap_time_sum
print lap_time_sum/60

In [ ]:
# Slicing arrays in python
foo = [0,1,2,3,4,5]
print foo[:2]
print foo[0:2]
print foo[3:5]


In [ ]:
def records_by_lap(messages, lap_no):
    lap_records=[]
    laps_found = 0
    last_idx = 0
    idx = 0
    lap_records.append([])
    for msg in messages:
        lap_records[laps_found].append(msg)
        if msg.name == 'lap':
            laps_found += 1
            lap_records.append([])
            print 'lap {} found at {}'.format(laps_found, idx)
            
        idx += 1
        
    return lap_records
        
    

In [ ]:
rbl = records_by_lap(messages, 1)
print len(rbl)

In [ ]:
print filter(lambda x:x != None, map(lambda x:x.get_value('heart_rate'), rbl[0]))

In [ ]:
import numpy as np
print int(round(np.average(filter(lambda x:x != None, map(lambda x:x.get_value('heart_rate'), rbl[0])))))

In [ ]:
import numpy as np

def average_val_for_field(records, field_name):
    fields = map(lambda x:x.get_value(field_name), records)
    fields_with_vals = filter(lambda x:x != None, fields)
    avg = int(round(np.average(fields_with_vals)))
    return avg

In [ ]:
print laps[0].get_value('avg_heart_rate')

In [ ]:
print average_val_for_field(rbl[0], 'heart_rate')
print average_val_for_field(rbl[0], 'power')

In [ ]:
import pandas as pd

In [ ]:
s = pd.Series(map(lambda x:x.get_value('power'), filter(lambda x:x.name=='record',rbl[0])))
print s

In [ ]:
print type(s)
s.describe()

In [ ]:
rolling_means = s.rolling(window=30,center=False).mean()[31:]
print type(rolling_means)

In [ ]:
from math import pow

rmp = rolling_means.map(lambda x:pow(x,4))

In [ ]:
print rmp.mean()

In [ ]:
np = rmp.mean()**(1/float(4))

In [ ]:
print int(round(np))
print laps[0].get_value('normalized_power')

In [ ]:
def extract_record_field(messages, field):
    #Get records
    records = filter(lambda m:m.name=='record', messages)
    fields = map(lambda r:r.get_value(field), records)
    return filter(lambda f:f != None, fields)

In [ ]:
power = extract_record_field(messages, 'power')
print power

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
n, bins, patches = plt.hist(power, 30, normed=0, facecolor='g', alpha=0.75)

print n
print bins
print patches

plt.xlabel('Power')
plt.ylabel('Time')
plt.title('Power Histogram')
plt.axis([0, 300, 0, 400])
plt.grid(True)
plt.show()

In [ ]:
hr = extract_record_field(messages, 'heart_rate')

plt.hist(hr, 30, normed=0, facecolor='g', alpha=0.75)

plt.xlabel('Heart Rate')
plt.ylabel('Time')
plt.title('Power Histogram')
plt.axis([50, 250, 0, 300])
plt.grid(True)
plt.show()

In [ ]:
def make_zone(name, low, high):
    zone = {}
    zone['Name'] = name
    zone['Low'] = low
    zone['High'] = high
    
    return zone

In [ ]:
em = make_zone('EnduranceMiles', 0.45, 0.73)
print em

In [ ]:
def calculate_zone(field_test_power, zone):
    czone = {}
    czone['Name'] = zone['Name']
    czone['Low'] = int(round(zone['Low']*field_test_power))
    czone['High'] = int(round(zone['High']*field_test_power))
    return czone

In [ ]:
print calculate_zone(213,em)
print em

In [ ]:
def power_training_intensities(field_test_power):
    i = {}
    
    